# Tanaka function multi-objective optimization (min).

Minimize the equations given by:

\begin{cases}
      f_{1}\left(\mathbf{x}\right) = x, \\
      f_{2}\left(\mathbf{x}\right) = y, \\
\end{cases}

subject to:

\begin{cases}
      C_{1}\left(\mathbf{x}\right) = x^{2} + y^{2} - 1 - 0.1\cos\left(16 \arctan(x/y) \right) \geq 0, \\
      C_{2}\left(\mathbf{x}\right) = \left(x - 0.5\right)^{2} + \left(y - 0.5\right)^{2} \leq 0.5 \\
\end{cases}

where:

\begin{cases}
      0\le x \le \pi, \\
      0\le y \le \pi. \\
\end{cases}

The constrained Pareto-optimal solutions lie on the boundary of the first constraint. Since the constraint function is periodic and the second constraint function must also be satisfied, not all solutions on the boundary of the first constraint are Pareto-optimal. 

### First we import python libraries and set up the directory of our code.

In [ ]:
import os, sys
import numpy as np
from math import fsum, isclose
import matplotlib.pyplot as plt

PROJECT_DIR = os.path.abspath('..')
sys.path.append(PROJECT_DIR)

### Here we import all our custom GA code.

In [ ]:
# Import main classes.
from pygenalgo.genome.gene import Gene
from pygenalgo.genome.chromosome import Chromosome
from pygenalgo.engines.standard_ga import StandardGA

# Import Selection Operator(s).
from pygenalgo.operators.selection.linear_rank_selector import LinearRankSelector

# Import Crossover Operator(s).
from pygenalgo.operators.crossover.uniform_crossover import UniformCrossover

# Import Mutation Operator(s).
from pygenalgo.operators.mutation.random_mutator import RandomMutator

### Define the multi-objective function, which plays also the role of the 'fitness' function.

In addition, we define the 'rand_fx' which takes the role of the 'random()' method of the Genes. Every 
time we want to 'mutate' a gene this function will be called that returns 'valid', but random values for 
the gene.

In [ ]:
# Multi-objective function.
def fun_Tanaka(individual: Chromosome, f_min: bool = True):
    
    # Set the penalty coefficient.
    p_coeff = 0.8
    
    # Extract genes from the chromosome.
    x1, x2 = [gene.datum for gene in individual.genome]
    
    # Compute each objective function.
    f1, f2 = x1, x2
    
    # Avoid "division by zero" errors.
    if isclose(x2, 0.0):
        x2 += 0.0001
    # _end_if_
    
    # Compute the constraints.
    C1 = min(0.0, x1**2 + x2**2 - 1.0 - 0.1*np.cos(16.0 * np.arctan(x1/x2)))**2
    C2 = max(0.0, (x1 - 0.5)**2 + (x2 - 0.5)**2 - 0.5)**2
    
    # Compute the final value.
    f_val = fsum([0.5*f1, 0.5*f2, p_coeff*C1, p_coeff*C2])
    
    # Return the negative (to account for minimization).
    return -f_val if f_min else f_val
# _end_def_

# Random functions:
rand_fx1 = lambda: np.random.uniform(0.0, np.pi)
rand_fx2 = lambda: np.random.uniform(0.0, np.pi)

Here we set the GA parameters, such as number of genes, number of chromosomes, etc. Note that in this case 
each gene has different random() function (set by 'rand_fx1' and 'rand_fx2'), because they are valid in 
different ranges according to the problem definition.

In [ ]:
# Define the number of chromosomes.
N = 100

# Initial population.
population = [Chromosome([Gene(np.random.uniform(0.0, np.pi), rand_fx1),
                          Gene(np.random.uniform(0.0, np.pi), rand_fx2)], np.nan, True)
              for i in range(N)]

# Create the StandardGA object that will carry on the optimization.
toy_GA = StandardGA(initial_pop=population,
                    fit_func=fun_Tanaka,
                    select_op=LinearRankSelector(),
                    mutate_op=RandomMutator(0.15),
                    cross_op=UniformCrossover(0.85))

### Optimization process.

Here we call the GA object (either directly, or through the method run()). We set a number of parameter, such as the maximum iterations (i.e. epochs), tolerance for the fitness convergences, etc.

In [ ]:
toy_GA(epochs=5000, elitism=True, f_tol=1.0e-6, verbose=False)

In [ ]:
x1, x2 = [gene.datum for gene in toy_GA.best_chromosome().genome]

f1, f2 = x1, x2

print(f"x1={x1:.5f}, x2={x2:.5f}", end='\n\n')
print(f"f1(x1, x2) = {f1:.5f}")
print(f"f2(x1, x2) = {f2:.5f}")

**Note that the above solution is indeed in the Pareto-optimal solutions!**

### End of file